프로젝트: 한국어 데이터로 챗봇 만들기
=======

어텐션을 사용하는 트랜스포머의 인코더 디코더 구조를 한국어로 이루어진 챗봇 데이터에 적용해 본다.

-21.11.23-

In [3]:
# 필요한 패키지 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

# 1. 데이터 수집하기

한국어 챗봇 데이터는 송영숙님이 공개한 챗봇 데이터를 사용합니다.    
https://github.com/songys/Chatbot_data/blob/master/ChatbotData.csv

In [2]:
!ls ~/aiffel/Exploration/E15/data

'ChatbotData .csv'
